In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Mesothelioma"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Mesothelioma/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Mesothelioma/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Mesothelioma/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Mesothelioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# Review subdirectories to find the most relevant match for Mesothelioma
all_dirs = os.listdir(tcga_root_dir)

# Print all available directories for debugging
print("Available TCGA directories:")
print(all_dirs)

# Looking for directories related to our target trait
trait_related_dirs = [d for d in all_dirs if trait.lower() in d.lower()]

if len(trait_related_dirs) > 0:
    # If we found related directories, choose the most specific one
    selected_dir = trait_related_dirs[0]
    selected_path = os.path.join(tcga_root_dir, selected_dir)
    
    # Get paths to the clinical and genetic data files
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(selected_path)
    
    # Load the data files
    clinical_data = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
    genetic_data = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
    
    # Print the column names of the clinical data
    print("Clinical data columns:")
    print(clinical_data.columns.tolist())
    
    # Also print basic information about both datasets
    print("\nClinical data shape:", clinical_data.shape)
    print("Genetic data shape:", genetic_data.shape)
    
    # Set flags for validation
    is_gene_available = genetic_data.shape[0] > 0
    is_trait_available = clinical_data.shape[0] > 0
else:
    print(f"No directories found related to {trait} in the TCGA dataset.")
    
    # Mark this task as completed with no suitable directory found
    is_gene_available = False
    is_trait_available = False
    validate_and_save_cohort_info(
        is_final=False, 
        cohort="TCGA", 
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Available TCGA directories:
['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Cancer_

Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'asbestos_exposure_age', 'asbestos_exposure_age_last', 'asbestos_exposure_source', 'asbestos_exposure_type', 'asbestos_exposure_years', 'assessment_timepoint_category', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'creatinine_norm_range_lower', 'creatinine_norm_range_upper', 'creatinine_prior_tx', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'eastern_cancer_oncology_group', 'family_history_cancer_type', 'family_history_cancer_type_other', 'family_member_relationship_type', 'form_completion_date', 'gender', 'histological_type', 'history_asbestos_exposure', 'history_of_neoadjuvant_treatment', 'icd_10', 'icd_o_3_histology', 'icd_o_3_sit

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify potential age-related columns
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'asbestos_exposure_age', 
                      'asbestos_exposure_age_last', 'days_to_birth']

# Identify potential gender-related columns
candidate_gender_cols = ['gender']

# Let's first load the data for the cohort
cohort_dir = os.path.join(tcga_root_dir, "TCGA_Mesothelioma_(MESO)")
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Try reading the file with tab delimiter which is common in TCGA files
try:
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
except Exception as e:
    print(f"Error with tab delimiter: {e}")
    # If that fails, try the python engine which can detect the delimiter
    try:
        clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep=None, engine='python')
    except Exception as e:
        print(f"Error with python engine: {e}")
        # As a last resort, try various common delimiters
        for sep in [',', ';', '|']:
            try:
                clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep=sep)
                print(f"Successfully read with delimiter: '{sep}'")
                break
            except:
                continue

# Extract and preview candidate age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

# Extract and preview candidate gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

# Display the previews
print("Candidate age columns:")
print(candidate_age_cols)
print("\nAge columns preview:")
print(age_preview)

print("\nCandidate gender columns:")
print(candidate_gender_cols)
print("\nGender columns preview:")
print(gender_preview)


Candidate age columns:
['age_at_initial_pathologic_diagnosis', 'asbestos_exposure_age', 'asbestos_exposure_age_last', 'days_to_birth']

Age columns preview:
{'age_at_initial_pathologic_diagnosis': [64, 60, 53, 58, 69], 'asbestos_exposure_age': [nan, nan, nan, 25.0, nan], 'asbestos_exposure_age_last': [nan, nan, nan, 31.0, nan], 'days_to_birth': [-23591, -21972, -19503, -21423, -25471]}

Candidate gender columns:
['gender']

Gender columns preview:
{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Examine the candidate age columns
# age_at_initial_pathologic_diagnosis: Contains actual ages (seems most direct)
# asbestos_exposure_age and asbestos_exposure_age_last: Many NaN values
# days_to_birth: Contains negative values (days before birth, would need conversion)

# Examine the candidate gender column
# gender: Contains clear MALE/FEMALE values

# Select the appropriate columns
age_col = "age_at_initial_pathologic_diagnosis"  # Most direct representation of patient age
gender_col = "gender"  # Only candidate and contains proper values

# Print the selected columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Let's use the correct folder for Mesothelioma data
selected_dir = "TCGA_Mesothelioma_(MESO)"
selected_path = os.path.join(tcga_root_dir, selected_dir)

# Reload the clinical and genetic data files to ensure we're using the correct dataset
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(selected_path)
clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')

# 1. Extract and standardize clinical features (trait, age, gender)
selected_clinical_df = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# 2. Normalize gene symbols in gene expression data
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")
print(f"Gene data shape after normalization: {normalized_gene_df.shape}")

# 3. Link clinical and genetic data
# Transpose the genetic data to have samples as rows
genetic_df_t = normalized_gene_df.T
# Ensure the indices match between datasets
common_samples = list(set(genetic_df_t.index) & set(selected_clinical_df.index))
genetic_df_filtered = genetic_df_t.loc[common_samples]
clinical_df_filtered = selected_clinical_df.loc[common_samples]

# Combine the datasets
linked_data = pd.concat([clinical_df_filtered, genetic_df_filtered], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data_cleaned.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data_filtered = judge_and_remove_biased_features(linked_data_cleaned, trait)

# 6. Validate data quality and save cohort information
# First check if gene and trait data are available
is_gene_available = linked_data_filtered.shape[1] > 3  # More columns than just trait, age, gender
is_trait_available = trait in linked_data_filtered.columns

# Second validation for saving metadata
note = f"Dataset contains {linked_data_filtered.shape[0]} samples and {linked_data_filtered.shape[1] - 3} genes after preprocessing."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data_filtered,
    note=note
)

# 7. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_filtered.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
    
    # Also save the clinical data separately
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_cols = [col for col in linked_data_filtered.columns if col in [trait, 'Age', 'Gender']]
    linked_data_filtered[clinical_cols].to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Dataset not usable for analysis. Data not saved.")

Normalized gene data saved to ../../output/preprocess/Mesothelioma/gene_data/TCGA.csv
Gene data shape after normalization: (19848, 87)
Linked data shape: (87, 19851)


Data shape after handling missing values: (87, 19851)
Quartiles for 'Mesothelioma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Mesothelioma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 57.0
  50% (Median): 64.0
  75%: 69.0
Min: 28
Max: 81
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 16 occurrences. This represents 18.39% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset not usable for analysis. Data not saved.
